# The prediction accuracy of subscribing to a game-related newsletter using gender, age, and played hours.

## Introduction

* This project will mainly focus on a dataset about how people play video games. It will use data from this dataset to calculate the prediction accuracy of different characteristics or behaviors. The goal is to identify which characteristics or behaviors are more predictive. In the future, these characteristics can be used to predict whether a player will subscribe to a game-related newsletter.

  
* According to the dataset provided by a research group in Computer Science at UBC, we will primarily analyze three characteristics and behaviors. The main research question of this project is: "Among gender, age, and hours played, which factor has the highest prediction accuracy for subscribing to a game-related newsletter?"